# Creating Machine learning Models.

### Importing Libraries

In [1]:
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from imblearn.combine import SMOTEENN

### Importing CSV


In [2]:
df = pd.read_csv(r"C:\Users\atewa\OneDrive\Desktop\DA_Projects\Customer Churn Analysis\tdf_dummies.csv")
df.head()

,Unnamed: 0,SeniorCitizen,MonthlyCharges,TotalCharges,Churn,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,PhoneService_No,PhoneService_Yes,MultipleLines_No,MultipleLines_No phone service,MultipleLines_Yes,InternetService_DSL,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No,OnlineSecurity_No internet service,OnlineSecurity_Yes,OnlineBackup_No,OnlineBackup_No internet service,OnlineBackup_Yes,DeviceProtection_No,DeviceProtection_No internet service,DeviceProtection_Yes,TechSupport_No,TechSupport_No internet service,TechSupport_Yes,StreamingTV_No,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_1 - 12,tenure_group_13 - 24,tenure_group_25 - 36,tenure_group_37 - 48,tenure_group_49 - 60,tenure_group_61 - 72
0,0,0,29.85,29.85,0,1,0,0,1,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0
1,1,0,56.95,1889.50,0,0,1,1,0,1,0,0,1,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0
2,2,0,53.85,108.15,1,0,1,1,0,1,0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0
3,3,0,42.30,1840.75,0,0,1,1,0,1,0,1,0,0,1,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,1,0,1,0,1,0,0,0,0,0,0,1,0,0
4,4,0,70.70,151.65,1,1,0,1,0,1,0,0,1,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0


In [3]:
df = df.drop('Unnamed: 0',axis=1)

In [4]:
X = df.drop('Churn', axis=1)
X.head()

,SeniorCitizen,MonthlyCharges,TotalCharges,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,PhoneService_No,PhoneService_Yes,MultipleLines_No,MultipleLines_No phone service,MultipleLines_Yes,InternetService_DSL,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No,OnlineSecurity_No internet service,OnlineSecurity_Yes,OnlineBackup_No,OnlineBackup_No internet service,OnlineBackup_Yes,DeviceProtection_No,DeviceProtection_No internet service,DeviceProtection_Yes,TechSupport_No,TechSupport_No internet service,TechSupport_Yes,StreamingTV_No,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_1 - 12,tenure_group_13 - 24,tenure_group_25 - 36,tenure_group_37 - 48,tenure_group_49 - 60,tenure_group_61 - 72
0,0,29.85,29.85,1,0,0,1,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0
1,0,56.95,1889.50,0,1,1,0,1,0,0,1,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0
2,0,53.85,108.15,0,1,1,0,1,0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0
3,0,42.30,1840.75,0,1,1,0,1,0,1,0,0,1,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,1,0,1,0,1,0,0,0,0,0,0,1,0,0
4,0,70.70,151.65,1,0,1,0,1,0,0,1,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0


In [5]:
y=df['Churn']
y.head()

0    0
1    0
2    1
3    0
4    1
Name: Churn, dtype: int64

### Train Test Split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

### Decision Tree Classifier

In [7]:
mdt = DecisionTreeClassifier(criterion="gini", random_state=100, max_depth=6, min_samples_leaf=8)

In [9]:
mdt.fit(X_train,y_train)

DecisionTreeClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [10]:
y_pr = mdt.predict(X_test)
y_pr

array([0, 0, 1, ..., 0, 1, 0])

In [11]:
mdt.score(X_test,y_test)

0.7746979388770433

In [12]:
print(classification_report(y_test,y_pr, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.83      0.87      0.85      1023
           1       0.60      0.52      0.56       384

    accuracy                           0.77      1407
   macro avg       0.71      0.70      0.70      1407
weighted avg       0.77      0.77      0.77      1407



##### As you can see that the accuracy is quite low, and as it's an imbalanced dataset, we shouldn't consider Accuracy as our metrics to measure the model, as Accuracy is cursed in imbalanced datasets.

##### Hence, we need to check recall, precision & f1 score for the minority class, and it's quite evident that the precision, recall & f1 score is too low for Class 1, i.e. churned customers.

##### Hence, moving ahead to call SMOTEENN (UpSampling + ENN)

In [14]:
sm = SMOTEENN()
X_resampled, y_resampled = sm.fit_resample(X,y)

In [15]:
xr_train,xr_test,yr_train,yr_test=train_test_split(X_resampled, y_resampled,test_size=0.2)

In [18]:
mdt_smote=DecisionTreeClassifier(criterion = "gini",random_state = 100,max_depth=6, min_samples_leaf=8)

In [19]:
mdt_smote.fit(xr_train,yr_train)
yr_predict = mdt_smote.predict(xr_test)
model_score_r = mdt_smote.score(xr_test, yr_test)
print(model_score_r)
print(metrics.classification_report(yr_test, yr_predict))

0.9263959390862944
              precision    recall  f1-score   support

           0       0.93      0.91      0.92       530
           1       0.93      0.94      0.93       652

    accuracy                           0.93      1182
   macro avg       0.93      0.92      0.93      1182
weighted avg       0.93      0.93      0.93      1182



In [20]:
print(metrics.confusion_matrix(yr_test, yr_predict))

[[482  48]
 [ 39 613]]


##### Now we can see quite better results, i.e. Accuracy: 92 %, and a very good recall, precision & f1 score for minority class.

##### Let's try with some other classifier.

### Random Forest Classifier

In [21]:
from sklearn.ensemble import RandomForestClassifier

In [22]:
mdrf = RandomForestClassifier(n_estimators=100,max_depth=6, min_samples_leaf=8)

In [23]:
mdrf.fit(X_train,y_train)

RandomForestClassifier(max_depth=6, min_samples_leaf=8)

In [24]:
y_pr = mdrf.predict(X_test)
y_pr

array([0, 0, 1, ..., 0, 0, 0])

In [25]:
mdrf.score(X_test,y_test)

0.8045486851457001

In [26]:
print(classification_report(y_test, y_pr, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.82      0.93      0.87      1023
           1       0.72      0.47      0.57       384

    accuracy                           0.80      1407
   macro avg       0.77      0.70      0.72      1407
weighted avg       0.79      0.80      0.79      1407



In [27]:
sm = SMOTEENN()
X_resampled1, y_resampled1 = sm.fit_resample(X,y)

In [28]:
xr_train1,xr_test1,yr_train1,yr_test1=train_test_split(X_resampled1, y_resampled1,test_size=0.2)

In [29]:
mdrf_smote=RandomForestClassifier(n_estimators=100, criterion='gini', random_state = 100,max_depth=6, min_samples_leaf=8)

In [30]:
mdrf_smote.fit(xr_train1,yr_train1)

RandomForestClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [31]:
yr_predict1 = mdrf_smote.predict(xr_test1)

In [32]:
model_score_r1 = mdrf_smote.score(xr_test1, yr_test1)

In [33]:
print(model_score_r1)
print(metrics.classification_report(yr_test1, yr_predict1))

0.9331619537275064
              precision    recall  f1-score   support

           0       0.95      0.90      0.92       516
           1       0.92      0.96      0.94       651

    accuracy                           0.93      1167
   macro avg       0.93      0.93      0.93      1167
weighted avg       0.93      0.93      0.93      1167



In [34]:
print(metrics.confusion_matrix(yr_test1, yr_predict1))

[[465  51]
 [ 27 624]]


##### With RF Classifier, also we are able to get quite good results, infact better than Decision Tree.


### Performing PCA

In [35]:
from sklearn.decomposition import PCA
pca = PCA(0.9)
xr_train_pca = pca.fit_transform(xr_train1)
xr_test_pca = pca.transform(xr_test1)
explaned_variance = pca.explained_variance_ratio_

In [36]:
md = RandomForestClassifier(n_estimators=100, criterion='gini', random_state=100, max_depth=6, min_samples_leaf=8)

In [37]:
md.fit(xr_train_pca,yr_train1)

RandomForestClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [38]:
yr_predict_pca = md.predict(xr_test_pca)

In [39]:
model_score_r_pca = md.score(xr_test_pca, yr_test1)

In [40]:
print(model_score_r_pca)
print(metrics.classification_report(yr_test1, yr_predict_pca))

0.7506426735218509
              precision    recall  f1-score   support

           0       0.74      0.67      0.70       516
           1       0.76      0.81      0.78       651

    accuracy                           0.75      1167
   macro avg       0.75      0.74      0.74      1167
weighted avg       0.75      0.75      0.75      1167



#### With PCA, we couldn't see any better results, hence let's finalise the model which was created by RF Classifier

### Pickling the model

In [41]:
import pickle

In [44]:
filename = r'C:\Users\atewa\OneDrive\Desktop\DA_Projects\Customer Churn Analysis\model.sav'
pickle.dump(mdrf_smote, open(filename,'wb'))

In [45]:
load_model = pickle.load(open(filename, 'rb'))

In [46]:
model_score_r1 = load_model.score(xr_test1, yr_test1)

In [47]:
model_score_r1 = load_model.score(xr_test1,yr_test1)

In [48]:
model_score_r1

0.9331619537275064